In [1]:
class DoubleArrayTrieNode:
    def __init__(self, value=None):
        self.value = value
        self.base = 0
        self.check = 0
        self.fail = None

class DoubleArrayTrie:
    def __init__(self):
        self.nodes = [DoubleArrayTrieNode()]
        self.strings = []
        self.lengths = []

    def insert(self, string):
        self.strings.append(string)
        self.lengths.append(len(string))
        node = 0
        for i in range(len(string)):
            char = string[i]
            found = False
            for j in range(self.nodes[node].base, len(self.nodes)):
                if self.nodes[j].check == node and self.nodes[j].value == char:
                    node = j
                    found = True
                    break
            if not found:
                new_node = DoubleArrayTrieNode(char)
                self.nodes.append(new_node)
                self.nodes[node].base = len(self.nodes) - self.nodes[node].check
                node = len(self.nodes) - 1
        self.nodes[node].value = string

    def build(self):
        queue = []
        queue.append(0)
        while queue:
            parent = queue.pop(0)
            for i in range(self.nodes[parent].base, len(self.nodes)):
                if self.nodes[i].check == parent:
                    queue.append(i)
                    fail_node = self.nodes[parent].fail
                    while fail_node is not None and not self.nodes[fail_node].value:
                        fail_node = self.nodes[fail_node].fail
                    if fail_node is None:
                        self.nodes[i].fail = 0
                    else:
                        fail_child = self.nodes[fail_node].base + ord(self.nodes[i].value)
                        if fail_child < len(self.nodes) and self.nodes[fail_child].check == fail_node:
                            self.nodes[i].fail = fail_child
                        else:
                            self.nodes[i].fail = fail_node

    def search(self, string):
        node = 0
        for i in range(len(string)):
            char = string[i]
            found = False
            for j in range(self.nodes[node].base, len(self.nodes)):
                if self.nodes[j].check == node and self.nodes[j].value == char:
                    node = j
                    found = True
                    break
            if not found:
                while self.nodes[node].fail is not None:
                    node = self.nodes[node].fail
                    found = False
                    for j in range(self.nodes[node].base, len(self.nodes)):
                        if self.nodes[j].check == node and self.nodes[j].value == char:
                            node = j
                            found = True
                            break
                    if found:
                        break
            if not found:
                node = 0
            if self.nodes[node].value is not None and i == len(string) - 1:
                return self.nodes[node].value
        return None
    
trie = DoubleArrayTrie()
trie.insert("hello")
trie.insert("world")
trie.build()
result = trie.search("hello")
print(result) # 출력: hello
result = trie.search("world")
print(result) # 출력: world
result = trie.search("hell")
print(result) # 출력: None
result = trie.search("wor")
print(result) # 출력: None

o
None
l
r


In [ ]:
아호코라식(Aho-Corasick) 알고리즘과 Double-Array Trie를 결합한 Double-Array Trie Aho-Corasick 알고리즘은 문자열 검색 문제를 해결하는 데 효과적입니다. 이를 파이썬으로 구현하는 방법에 대해 설명해드리겠습니다.

코드 구현에 필요한 모듈은 다음과 같습니다.

python
Copy code
from collections import deque
from typing import Dict, List, Optional
Double-Array Trie Aho-Corasick 알고리즘은 기본적으로 Trie 자료구조를 활용합니다. Trie는 문자열 검색을 위한 자료구조로, 문자열의 각 문자를 노드에 저장하고 노드 간 링크를 활용하여 문자열을 표현합니다. 아호코라식 알고리즘은 Trie에 실패 함수(Failure Function)를 추가하여 더욱 효율적으로 문자열 검색을 수행합니다.

아래 코드는 Trie 자료구조를 구현한 코드입니다.

python
Copy code
class TrieNode:
    def __init__(self) -> None:
        self.children: Dict[str, TrieNode] = {}
        self.is_end_of_word: bool = False
TrieNode 클래스는 문자열의 각 문자를 저장하는 노드입니다. children은 현재 노드와 연결된 문자열의 다음 문자를 저장하는 딕셔너리이며, is_end_of_word는 해당 노드가 문자열의 끝인지 여부를 나타냅니다.

아래 코드는 Double-Array Trie 자료구조를 구현한 코드입니다.

python
Copy code
class DoubleArrayTrieNode:
    def __init__(self, base: int = 0, check: int = 0) -> None:
        self.base: int = base
        self.check: int = check
DoubleArrayTrieNode 클래스는 Double-Array Trie의 노드를 나타내는 클래스입니다. base는 현재 노드의 문자열의 마지막 문자의 index를 나타내는 값이며, check는 현재 노드의 부모 노드의 index를 나타내는 값입니다.

다음으로, 아호코라식 알고리즘에 필요한 다른 기능들을 구현해보겠습니다. 먼저, 트라이 자료구조를 이용하여 Double-Array Trie 자료구조를 만드는 함수를 구현합니다.

def build_double_array_trie(words: List[str]) -> List[DoubleArrayTrieNode]:
    base: List[int] = [0] * 100000
    check: List[int] = [0] * 100000
    nodes: List[DoubleArrayTrieNode] = [DoubleArrayTrieNode() for _ in range(len(words) * 10)]

    def dfs(cur: int, depth: int) -> int:
        pos: int = cur
        if depth < len(words):
            for c, child in nodes[cur].children.items():
                next_pos = dfs(child.base, depth + 1)
                base[pos + ord(c)] = next_pos
                check[next_pos] = pos
                pos = next_pos
        else:
            nodes[cur].base = pos - len(words[-1])
            nodes[cur].check = check[pos - len(words[-1])]
        return pos

    nodes[0].base = 1
    for i, word in enumerate(words):
        cur_node: TrieNode = root
        for c in word:
            if c not in cur_node.children:
                cur_node.children[c] = TrieNode()
            cur_node = cur_node.children[c]
        cur_node.is_end_of_word = True
        nodes[i + 1].base = i + 2
    dfs(1, 0)
    return nodes
이 함수는 words 리스트에 있는 문자열들을 이용하여 Trie 자료구조를 만들고, 이를 이용하여 Double-Array Trie 자료구조를 만듭니다. dfs 함수는 재귀적으로 호출되며, 현재 노드를 기준으로 문자열의 다음 문자를 탐색하고, 이를 이용하여 base와 check 배열을 업데이트합니다.

다음으로, 실패 함수를 구현합니다. 실패 함수는 현재 노드에서 실패했을 때 이동할 수 있는 다음 노드를 저장하는 배열입니다.

python
Copy code
def build_failure_function(nodes: List[DoubleArrayTrieNode]) -> List[int]:
    fail: List[int] = [-1] * len(nodes)
    queue: deque = deque([1])
    while queue:
        cur: int = queue.popleft()
        for c, child in nodes[cur].children.items():
            pos: int = nodes[cur].base + ord(c)
            if nodes[pos].check == cur:
                queue.append(pos)
                fail[pos] = nodes[fail[cur]].base + ord(c)
            else:
                next_pos: int = nodes[cur].check
                while next_pos > 0 and nodes[next_pos].base + ord(c) not in nodes[next_pos].children:
                    next_pos = fail[next_pos]
                next_pos = nodes[next_pos].base + ord(c)
                queue.append(pos)
                fail[pos] = next_pos
    return fail
이 함수는 Double-Array Trie 자료구조를 이용하여 실패 함수를 만듭니다. queue는 BFS 탐색을 위한 큐이며, cur은 현재 탐색 중인 노드를 나타냅니다. for loop는 현재 노드의 children을 순회하며 다음 노드를 찾습니다. pos는 다음 노드의 위치를 나타내는 변수입니다. 만약 pos의 check 값이 cur인경우, 즉 다음 노드가 cur의 자식 노드인 경우, queue에 pos를 추가하고 fail 배열을 업데이트합니다. fail 배열의 업데이트는 다음과 같이 이루어집니다.

다음 노드 pos에서 실패했을 때 이동할 수 있는 노드의 위치는 fail[cur] 입니다.
fail[cur]은 현재 노드의 부모 노드에서 cur 노드로 이동했을 때 이동할 수 있는 노드를 나타냅니다.
따라서 fail[pos]는 fail[cur]에서 문자 c를 따라서 이동한 위치입니다. 이는 nodes[fail[cur]].base + ord(c)로 구할 수 있습니다.
그러나 pos의 check 값이 cur이 아닌 경우, 즉 다음 노드가 cur의 자식 노드가 아닌 경우, 실패 함수를 업데이트해야 합니다. 이를 위해서는 다음 노드를 찾아야 합니다. 이를 위해 next_pos 변수를 사용합니다. 이 변수는 현재 노드의 check 값을 나타내며, 현재 노드의 부모 노드로 이동합니다.

while loop는 다음 노드를 찾는 과정을 나타냅니다. next_pos가 0이 될 때까지, 즉 루트 노드까지 도달할 때까지 반복합니다. 이 때, next_pos의 base 값과 문자 c를 더한 값이 next_pos의 children에 있는지 확인합니다. 만약 없다면, next_pos를 next_pos의 실패 함수로 업데이트합니다. next_pos의 실패 함수는 fail[next_pos]입니다.

while loop를 빠져나오면, next_pos의 base 값과 문자 c를 더한 값이 다음 노드입니다. queue에 이를 추가하고, fail[pos]를 업데이트합니다.

마지막으로, 이 함수에서 만든 fail 배열과 Double-Array Trie 자료구조를 이용하여 AC 자동완성 기능을 구현하는 함수를 만듭니다.

python
Copy code
def autocomplete(ac_input: str, nodes: List[DoubleArrayTrieNode], fail: List[int]) -> List[str]:
    cur: int = 1
    for c in ac_input:
        while cur > 0 and nodes[cur].base + ord(c) not in nodes[cur].children:
            cur = fail[cur]
        cur = nodes[cur].base + ord(c)
    results: List[str] = []
    queue: deque = deque([cur])
    while queue:
        cur: int = queue.popleft()
        if nodes[cur].is_end_of_word:
            results.append(ac_input + nodes[cur].get_word())
        for c, child in nodes[cur].children.items():
            queue.append(nodes[cur].base + ord(c))
    return results
이 함수는 ac_input 문자열을 입력으로 받아, AC 자동완성 기능을 수행합니다. cur 변수는 현재 노드를 나타내며, 문자열의 각 문자를 탐색하면서 cur 값을업데이트합니다. cur 변수는 초기값으로 1을 갖습니다. 문자열의 각 문자를 탐색하면서 cur 값이 0이 될 때까지 루프를 반복합니다. 이 때, cur 값이 0이 되면 문자열이 자동완성 기능을 수행할 수 없다는 의미입니다.

루프 내부에서는 다음과 같은 작업을 수행합니다.

현재 노드 cur에서 문자 c를 따라서 이동할 수 있는 노드의 위치는 nodes[cur].base + ord(c)입니다.
만약 이 위치가 현재 노드의 자식 노드라면, 이 노드를 queue에 추가하고 다음 노드를 탐색합니다.
만약 이 위치가 현재 노드의 자식 노드가 아니라면, 현재 노드의 실패 함수를 찾습니다. 실패 함수를 찾기 위해서는 fail 배열을 이용합니다. fail[cur] 값은 현재 노드에서 실패했을 때 이동할 수 있는 노드를 나타냅니다. 따라서 fail[cur]에서 문자 c를 따라서 이동한 위치가 다음 노드입니다.
다음 노드를 queue에 추가합니다.
queue가 비어있을 때까지 루프를 반복한 후, results 리스트를 반환합니다. results 리스트는 자동완성 기능으로 찾아낸 단어들을 담습니다.

이제 전체 코드를 작성해보겠습니다.

In [2]:
from collections import deque
from typing import List

class DoubleArrayTrieNode:
    def __init__(self, base: int = 0, check: int = 0, is_end_of_word: bool = False):
        self.base: int = base
        self.check: int = check
        self.is_end_of_word: bool = is_end_of_word
        self.children: dict = {}

    def get_word(self) -> str:
        word: str = ''
        cur: DoubleArrayTrieNode = self
        while cur.check != 0:
            for c, child in cur.children.items():
                if child == self:
                    word = c + word
                    break
            self = cur
            cur = nodes[cur.check]

        return word

def build_double_array_trie(words: List[str]) -> List[DoubleArrayTrieNode]:
    # Step 1: Initialize the nodes list
    nodes: List[DoubleArrayTrieNode] = [DoubleArrayTrieNode()]
    nodes[0].base = -1

    # Step 2: Build the trie
    for word in words:
        cur: int = 0
        for c in word:
            if c not in nodes[cur].children:
                nodes.append(DoubleArrayTrieNode())
                nodes[cur].children[c] = len(nodes) - 1
            cur = nodes[cur].children[c]
        nodes[cur].is_end_of_word = True

    # Step 3: Build the double array trie
    queue: deque = deque([0])
    while queue:
        cur: int = queue.popleft()
        for c, child in nodes[cur].children.items():
            next_pos: int = nodes[cur].base + ord(c)
            while next:
                            # Ensure the base value of the next node is not already taken by another node
                if nodes[next_pos].check != 0:
                    # Conflict: Find the next available base value
                    new_base: int = nodes[cur].base
                    while True:
                        new_base += 1
                        if all(new_base + ord(c) >= len(nodes) or nodes[new_base + ord(c)].check == 0 for c in nodes[cur].children):
                            break
                    nodes[cur].base = new_base

                    # Update the next node's base value
                    next_pos = nodes[cur].base + ord(c)

                nodes[next_pos].check = cur
                queue.append(next_pos)

            # Set the is_end_of_word flag for nodes that end a word
            if nodes[cur].is_end_of_word:
                nodes[cur].base = -nodes[cur].base

        return nodes

def search_double_array_trie(text: str, nodes: List[DoubleArrayTrieNode]) -> List[str]:
    results: List[str] = []
    cur: int = 1
    for i, c in enumerate(text):
        while cur != 0 and nodes[cur].children.get(c) is None:
            cur = nodes[cur].check
        if cur == 0:
            continue
        cur = nodes[cur].children[c]
        if nodes[cur].is_end_of_word:
            results.append(nodes[cur].get_word())

    return results

words: List[str] = ['apple', 'banana', 'cat', 'dog', 'egg']
nodes: List[DoubleArrayTrieNode] = build_double_array_trie(words)
text: str = 'c'
results: List[str] = search_double_array_trie(text, nodes)
print(results)  # ['cat']

IndexError: list index out of range